In [57]:
# Please execute/shift-return this cell everytime you run the notebook.  Don't edit it. 
%load_ext autoreload
%autoreload 2
from notebook import * 

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Memory hierarchy

## How much time did I spent on "memory"

Remember our demo long time ago?

In [ ]:
compare([do_render_code("./madd/madd_A.c", lang="c++", show=["//START", "//END"]),do_render_code("./madd/madd_B.c", lang="c++", show=["//START", "//END"])])

In [ ]:
render_code("./mv/matvec.c", lang="c++", show=["//START", "//END"])

In [ ]:
! cd mv; make clean; make; ./matvec 16 1 

In [ ]:
!echo "element,address"> addresses.csv; ./mv/matvec 16 1 >> addresses.csv
df = render_csv("addresses.csv")
df["tag"] = int(df["address"])
display_df_mono(df)

## The memory hierarchy of my computer

In [51]:
! lscpu | grep "Model name"
! getconf -a | grep CACHE

Model name:                      AMD Ryzen 7 5700G with Radeon Graphics
LEVEL1_ICACHE_SIZE                 32768
LEVEL1_ICACHE_ASSOC                8
LEVEL1_ICACHE_LINESIZE             64
LEVEL1_DCACHE_SIZE                 32768
LEVEL1_DCACHE_ASSOC                8
LEVEL1_DCACHE_LINESIZE             64
LEVEL2_CACHE_SIZE                  524288
LEVEL2_CACHE_ASSOC                 8
LEVEL2_CACHE_LINESIZE              64
LEVEL3_CACHE_SIZE                  16777216
LEVEL3_CACHE_ASSOC                 0
LEVEL3_CACHE_LINESIZE              64
LEVEL4_CACHE_SIZE                  0
LEVEL4_CACHE_ASSOC                 0
LEVEL4_CACHE_LINESIZE              0


In [52]:
! ssh htseng@azelf "lscpu | grep 'Model name'; getconf -a | grep CACHE"

Model name:                      12th Gen Intel(R) Core(TM) i9-12900KF
LEVEL1_ICACHE_SIZE                 65536
LEVEL1_ICACHE_ASSOC                8
LEVEL1_ICACHE_LINESIZE             64
LEVEL1_DCACHE_SIZE                 32768
LEVEL1_DCACHE_ASSOC                8
LEVEL1_DCACHE_LINESIZE             64
LEVEL2_CACHE_SIZE                  2097152
LEVEL2_CACHE_ASSOC                 16
LEVEL2_CACHE_LINESIZE              64
LEVEL3_CACHE_SIZE                  31457280
LEVEL3_CACHE_ASSOC                 12
LEVEL3_CACHE_LINESIZE              64
LEVEL4_CACHE_SIZE                  0
LEVEL4_CACHE_ASSOC                 0
LEVEL4_CACHE_LINESIZE              0


In [ ]:
! ssh htseng@nano "lscpu | grep 'Model name'; getconf -a | grep CACHE"

In [ ]:
! cd gpusort; make clean; make EXTRA_FLAGS=-pg 
! ssh htseng@azelf "cd courses/CS203/demo/amdahlslaw/gpusort; source ./run_CPU"

In [ ]:
! cd gpusort; gprof ./hybridsort_cpu ./gmon.out

In [ ]:
render_code("./gpusort/main.cu", lang="c++", show="bitonic_sort")

In [ ]:
! ssh htseng@azelf "nvidia-smi -a"

In [ ]:
! cd gpusort; make clean; make
# ! ssh htseng@azelf "source ./courses/CS203/demo/amdahlslaw/gpusort/run_CPU"
! echo "File on H.D.D.; Sorting on GPU"
! ssh htseng@azelf "cd courses/CS203/demo/amdahlslaw/gpusort; source ./run 2>> sort.csv"
! echo "File on S.S.D.; Sorting on GPU"
! ssh htseng@azelf "cd courses/CS203/demo/amdahlslaw/gpusort; source ./run_SSD 2>> sort.csv"

In [ ]:
display_df_mono(render_csv("./gpusort/sort.csv"))

In [ ]:
! cd vmul; make clean; make
! echo "THREADS,CPUTIME,HOST2GPU,GPUTIME,GPU2HOST" > ./vmul/vmul.csv
! echo "CPU based vul"
! ssh htseng@azelf "time ./courses/CS203/demo/amdahlslaw/vmul/vmul 33554432 1 0 30 2>> ./courses/CS203/demo/amdahlslaw/vmul/vmul.csv"
! echo "GPU based vul"
### i stands for "How many iterations each thread performs 
### -- the larger the number, the fewer the parallelism
! for i in 1 2 4 8 16 32 64 128 256 512 1024 2048 4096 8192; do ssh htseng@azelf "time ./courses/CS203/demo/amdahlslaw/vmul/vmul 33554432 $i 1 30 2>> ./courses/CS203/demo/amdahlslaw/vmul/vmul.csv" ; done

In [ ]:
df = render_csv("./vmul/vmul.csv")
df["TOTAL"] = df["CPUTIME"] + df["HOST2GPU"] + df["GPUTIME"] + df["GPU2HOST"]
df = df.sort_values(by=["THREADS"], ascending=True)
display_df_mono(df)
plotPE(df=df, lines=True, what=[ ('THREADS', "TOTAL"), ('THREADS', "GPUTIME")], columns=2)

In [ ]:
from IPython.display import IFrame
IFrame("https://hub.escalab.org:8000/user/htseng/terminals/1", width="100%", height="400")

Now, let's revisit the optimized gpusort on this machine with different array size...

In [ ]:
! echo "Configuration,Size,ET,FileInput,CPU_Kernel,GPU_Kernel,Host2GPU,GPU2Host" > sort_small.csv
! echo "File on H.D.D"
! cd gpusort; source ./run_small 512 2>> ../sort_small.csv
! echo "File on S.S.D"
! cd gpusort; source ./run_small_SSD 512 2>> ../sort_small.csv
! echo "File on H.D.D"
! cd gpusort; source ./run_small 32768 2>> ../sort_small.csv
! echo "File on S.S.D"
! cd gpusort; source ./run_small_SSD 32768 2>> ../sort_small.csv
! echo "File on H.D.D"
! cd gpusort; source ./run_small 262144 2>> ../sort_small.csv
! echo "File on S.S.D"
! cd gpusort; source ./run_small_SSD 262144 2>> ../sort_small.csv
display_df_mono((render_csv("sort_small.csv")))

What can we observe here?

## Inference per second

Inference per second is frequently used for ML performance, is it a good "metric"? Let's try using Google's edge TPU and compare it with CPU performance!

In [ ]:
!ssh htseng@celebi "python3 /nfshome/htseng/ESCAL/pycoral/benchmarks/inference_benchmarks.py"